In [6]:
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt

from eth_rpc import PrivateKeyWallet
from emp_orderly.utils import from_address
from emp_orderly.onboarding import deposit
from emp_orderly.onboarding.faucet import request_testnet_funds
from emp_orderly_types import PerpetualAssetType, Interval, OrderType
from emp_orderly import (
    Strategy, EmpOrderly,
    crossover, plot_heatmaps,
    EMA, SMA, SLOPE, CHOP,
    EmpyrealOrderlySDK,
)
from emp_orderly_types import PerpetualAssetType, Interval, OrderType

load_dotenv()

False

In [7]:
wallet = PrivateKeyWallet.create_new()
orderly_id = from_address(wallet.address)

sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=orderly_id, is_testnet=True)

In [8]:
class SmaCross(Strategy):
    n1: int = 10
    n2: int = 40

    @classmethod
    def update_lags(cls, n1, n2):
        cls.n1 = n1
        cls.n2 = n2

    def init(self):
        close = self.data.close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy(size=0.5)
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell(size=0.5)


tester = EmpOrderly(
    cash=1000,
    commission=.0001,
    exclusive_orders=True,
    sdk=sdk,
)

# load strategy and data
tester.set_strategy(SmaCross)
await tester.load_data(
    lookback=14,
    interval=Interval.five_minute,
    asset=PerpetualAssetType.APT,
)

# backtest
tester.backtest()

Start                     2024-09-17 14:45:00
End                       2024-09-21 09:15:00
Duration                      3 days 18:30:00
Exposure Time [%]                   94.572217
Equity Final [$]                  1088.848387
Equity Peak [$]                   1101.802878
Return [%]                           8.884839
Buy & Hold Return [%]               19.378151
Return (Ann.) [%]                  7848.90185
Volatility (Ann.) [%]             2309.479093
Sharpe Ratio                         3.398559
Sortino Ratio                             inf
Calmar Ratio                      5640.837991
Max. Drawdown [%]                   -1.391443
Avg. Drawdown [%]                   -0.383318
Max. Drawdown Duration        1 days 01:10:00
Avg. Drawdown Duration        0 days 03:42:00
# Trades                                   19
Win Rate [%]                        31.578947
Best Trade [%]                         9.4126
Worst Trade [%]                     -1.117921
Avg. Trade [%]                    

In [9]:
# plot
# tester.plot(show_price_data=False)
# plt.show()